In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# 2. Install Required Libraries
!pip install transformers datasets evaluate

In [ ]:
# 3. Import Libraries

import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

import numpy as np
import evaluate

In [ ]:
# 4. Set Up Path

# Base directory in Google Drive to save models and data
base_dir = '/content/drive/MyDrive/movie_sentiment_model'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

In [ ]:
# 5. Load and Prepare the IMDb Dataset

# Load the IMDb dataset
raw_datasets = load_dataset('imdb')

# Prepare the tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=256,
    )

# Tokenize the datasets
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Remove unnecessary columns and set format
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets.set_format('torch')

In [ ]:
print("\nraw datasets:\n")
print(raw_datasets)

print("\ntokenized datasets:\n")
print(tokenized_datasets)

In [ ]:
# 6. prepare the training dataset

# Shuffle the train dataset for training
train_dataset = tokenized_datasets['train'].shuffle(seed=42)

eval_dataset = tokenized_datasets['test']

In [ ]:
# 7. Set Up the Evaluation Metric

metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# 8. Set up the Pre-trained model

# Determine the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load the model
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Move the model to the device
model.to(device)

# Set Up Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

# initialize the Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics,
)

In [ ]:
# 9. Start training

trainer.train()

In [ ]:
# 10. Evaluate the model on validation data

eval_result = trainer.evaluate()
print(f"Accuracy: {eval_result['eval_accuracy']:.4f}")

In [ ]:
# 11. Save the model

model.save_pretrained(base_dir)
tokenizer.save_pretrained(base_dir)